In [2]:
import math
import cv2
import mediapipe as mp
import time

mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()

count = 0
squat_started = False
last_pose = None


pTime = 0


health = 100


#스쿼트 카운트 함수
def detect_squats(frame):
    global count, squat_started, last_pose, health
    
    min_visibility = 0.9
    landmarks_23_to_31 = results.pose_landmarks.landmark[23:32]
    is_all_visible = all(landmark.visibility >= min_visibility for landmark in landmarks_23_to_31)


    if is_all_visible:
        
    
        #랜드마크 인덱스 설정
        landmarks = results.pose_landmarks.landmark
        
        left_hip = landmarks[mpPose.PoseLandmark.LEFT_HIP]
        left_knee = landmarks[mpPose.PoseLandmark.LEFT_KNEE]
        left_ankle = landmarks[mpPose.PoseLandmark.LEFT_ANKLE]
        
         # 25번과 26번 랜드마크 좌표 얻기
        landmark_25 = results.pose_landmarks.landmark[25]
        landmark_26 = results.pose_landmarks.landmark[26]
        
        # 랜드마크 좌표를 화면 좌표로 변환
        h, w, c = frame.shape
        x25, y25 = int(landmark_25.x * w), int(landmark_25.y * h)
        x26, y26 = int(landmark_26.x * w), int(landmark_26.y * h)

        # 25번과 26번 랜드마크 각도 계산
        angle_rad = math.atan2(y26 - y25, x26 - x25)
        angle_deg = math.degrees(angle_rad)
        

        
        

            # 스쿼트 감지 조건 설정
        hip_to_knee_distance = left_knee.y - left_hip.y
        knee_to_ankle_distance = left_ankle.y - left_knee.y

            # 스쿼트 동작을 감지했을 때

        if hip_to_knee_distance > 0.1 and knee_to_ankle_distance > 0.1 and angle_rad < 95 and angle_deg < 95:
            if not squat_started:
                count += 1
                health=health+20
                squat_started = True
        else:
            squat_started = False
        

            # 현재 포즈 저장
        last_pose = landmarks

    return count




cap = cv2.VideoCapture(0)


# 게임 창 생성
cv2.namedWindow("Game Window")

# 초기 게이지 값 설정
cv2.createTrackbar("Health", "Game Window", health, 500, lambda x: None)

# 이전 시간 초기화
prev_time = cv2.getTickCount()




while True:
    ret, frame = cap.read()
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(imgRGB)
    
    if not ret:
        break
        
    current_time = cv2.getTickCount()

    # 경과 시간 계산 (단위: ms)
    elapsed_time = (current_time - prev_time) / cv2.getTickFrequency() * 1000

    # 1초가 경과할 때마다 체력 감소
    if elapsed_time >= 1000:  # 1초 (1000ms) 경과
        health -= 5
        # 체력이 0 이하로 내려가면 종료
        if health <= 0:
            break
        cv2.setTrackbarPos("Health", "Game Window", health)
        prev_time = current_time  # 이전 시간 업데이트

    # 게이지 그리기
    cv2.rectangle(
        img=frame,
        pt1=(10, 10),
        pt2=(health, 30),
        color=(0, 255, 0),  # 초록색
        thickness=cv2.FILLED,
    )


    
    
    #print(results.pose_landmarks)
    
    
    if results.pose_landmarks:
        
        
        squat_count = detect_squats(frame)
        
        
        
        
        mpDraw.draw_landmarks(frame, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
        for id, lm in enumerate(results.pose_landmarks.landmark):
            h, w, c = frame.shape
            print(id, lm)
            cx, cy = int(lm.x * w), int(lm.y * h)
            cv2.circle(frame, (cx, cy), 5, (255, 0, 0), cv2.FILLED)

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    
    # 왼쪽 숫자
    #cv2.putText(img, str(int(fps)), (70, 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)
    cv2.putText(frame, f"Squat Count: {squat_count}", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Game Window", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

0 x: 0.5156756043434143
y: 0.13076567649841309
z: -1.0293798446655273
visibility: 0.9999270439147949

1 x: 0.5635094046592712
y: 0.0342562198638916
z: -0.9799443483352661
visibility: 0.9998528957366943

2 x: 0.5928159356117249
y: 0.03926897048950195
z: -0.9795061945915222
visibility: 0.9998284578323364

3 x: 0.6229547262191772
y: 0.045931458473205566
z: -0.9796377420425415
visibility: 0.9998472929000854

4 x: 0.4691251814365387
y: 0.03142976760864258
z: -0.9797256588935852
visibility: 0.9998877048492432

5 x: 0.4371362626552582
y: 0.03782975673675537
z: -0.9789919853210449
visibility: 0.9998723268508911

6 x: 0.4114071726799011
y: 0.04722785949707031
z: -0.9792724847793579
visibility: 0.9998987913131714

7 x: 0.6801484823226929
y: 0.10069942474365234
z: -0.6122179627418518
visibility: 0.9998151659965515

8 x: 0.36943063139915466
y: 0.1170738935470581
z: -0.5817297101020813
visibility: 0.9999415874481201

9 x: 0.5723149180412292
y: 0.23770250380039215
z: -0.885028064250946
visibility: 0

0 x: 0.49855273962020874
y: 0.16633698344230652
z: -1.1148853302001953
visibility: 0.9999105334281921

1 x: 0.5557883977890015
y: 0.06422717124223709
z: -1.0432006120681763
visibility: 0.9997901916503906

2 x: 0.5841754078865051
y: 0.06889083981513977
z: -1.043030023574829
visibility: 0.9997797012329102

3 x: 0.6113011240959167
y: 0.07537411153316498
z: -1.0419702529907227
visibility: 0.999730110168457

4 x: 0.4541938304901123
y: 0.06338794529438019
z: -1.0498321056365967
visibility: 0.9998520016670227

5 x: 0.42336225509643555
y: 0.06676547974348068
z: -1.0491621494293213
visibility: 0.999859631061554

6 x: 0.39890700578689575
y: 0.07096827030181885
z: -1.0497376918792725
visibility: 0.9998687505722046

7 x: 0.6704037189483643
y: 0.1229846179485321
z: -0.6219448447227478
visibility: 0.9997252225875854

8 x: 0.36898040771484375
y: 0.12499745935201645
z: -0.6257994174957275
visibility: 0.9999379515647888

9 x: 0.5591427683830261
y: 0.2603300213813782
z: -0.9556196928024292
visibility: 0

0 x: 0.5058332681655884
y: 0.1758340448141098
z: -1.3663904666900635
visibility: 0.999789297580719

1 x: 0.5609049797058105
y: 0.07944207638502121
z: -1.2847015857696533
visibility: 0.999452531337738

2 x: 0.5914055109024048
y: 0.08464152365922928
z: -1.2849446535110474
visibility: 0.9994716644287109

3 x: 0.6199501752853394
y: 0.09117050468921661
z: -1.2845582962036133
visibility: 0.9992213845252991

4 x: 0.46291446685791016
y: 0.07586465775966644
z: -1.282075047492981
visibility: 0.9996170997619629

5 x: 0.4346323609352112
y: 0.07952241599559784
z: -1.2821520566940308
visibility: 0.9996819496154785

6 x: 0.41184201836586
y: 0.08549095690250397
z: -1.2829868793487549
visibility: 0.9996618032455444

7 x: 0.6815646886825562
y: 0.13732801377773285
z: -0.7753879427909851
visibility: 0.9992468953132629

8 x: 0.3883604109287262
y: 0.14467644691467285
z: -0.7495044469833374
visibility: 0.9998558163642883

9 x: 0.5695183277130127
y: 0.27060624957084656
z: -1.1600483655929565
visibility: 0.999

0 x: 0.650290846824646
y: 0.05239333212375641
z: -1.845446228981018
visibility: 0.9940040707588196

1 x: 0.6904310584068298
y: -0.06241337209939957
z: -1.774009108543396
visibility: 0.9840179681777954

2 x: 0.7077364325523376
y: -0.058334831148386
z: -1.7741057872772217
visibility: 0.9858389496803284

3 x: 0.7248920202255249
y: -0.05440916866064072
z: -1.7744070291519165
visibility: 0.9766079187393188

4 x: 0.6327378153800964
y: -0.08888038992881775
z: -1.824295997619629
visibility: 0.9913684129714966

5 x: 0.6104289889335632
y: -0.096932552754879
z: -1.8245779275894165
visibility: 0.9942793250083923

6 x: 0.5878715515136719
y: -0.10491878539323807
z: -1.825911521911621
visibility: 0.9941038489341736

7 x: 0.6947595477104187
y: 0.008688081055879593
z: -1.1295219659805298
visibility: 0.9810794591903687

8 x: 0.5014344453811646
y: -0.029124664142727852
z: -1.3489080667495728
visibility: 0.9982812404632568

9 x: 0.647826611995697
y: 0.11887407302856445
z: -1.5800210237503052
visibility: 0

0 x: 0.5747342109680176
y: -0.05493303760886192
z: -0.55928635597229
visibility: 0.8914368152618408

1 x: 0.5857529640197754
y: -0.07989665865898132
z: -0.537638247013092
visibility: 0.8495489954948425

2 x: 0.5930942893028259
y: -0.07466331124305725
z: -0.5377359986305237
visibility: 0.8607462644577026

3 x: 0.6003503799438477
y: -0.06912627816200256
z: -0.5379197597503662
visibility: 0.8467311859130859

4 x: 0.5654941201210022
y: -0.08314189314842224
z: -0.5333852767944336
visibility: 0.8584933280944824

5 x: 0.559697151184082
y: -0.08015832304954529
z: -0.5334092378616333
visibility: 0.8724812269210815

6 x: 0.5527111887931824
y: -0.07710745930671692
z: -0.533613920211792
visibility: 0.8670024871826172

7 x: 0.6076185703277588
y: -0.04946407303214073
z: -0.3390498757362366
visibility: 0.8566270470619202

8 x: 0.5442323684692383
y: -0.05761178955435753
z: -0.31961628794670105
visibility: 0.8934844136238098

9 x: 0.587708592414856
y: -0.016445696353912354
z: -0.4776376783847809
visibi

...

0 x: 0.5868626236915588
y: -0.0679641142487526
z: -0.7523455023765564
visibility: 0.9981993436813354

1 x: 0.5957480669021606
y: -0.07543999701738358
z: -0.761741042137146
visibility: 0.9966253638267517

2 x: 0.5986851453781128
y: -0.07437113672494888
z: -0.7615841627120972
visibility: 0.9959788918495178

3 x: 0.6017261743545532
y: -0.07315125316381454
z: -0.7617598176002502
visibility: 0.9959895610809326

4 x: 0.585797131061554
y: -0.07777083665132523
z: -0.7597986459732056
visibility: 0.997373104095459

5 x: 0.5824064612388611
y: -0.07807303965091705
z: -0.7599559426307678
visibility: 0.9972711801528931

6 x: 0.5788275599479675
y: -0.07843150198459625
z: -0.7600644826889038
visibility: 0.9977476000785828

7 x: 0.6019670367240906
y: -0.06586547195911407
z: -0.6491901278495789
visibility: 0.9961715936660767

8 x: 0.566993772983551
y: -0.0742967501282692
z: -0.6394094824790955
visibility: 0.9979437589645386

9 x: 0.5835077166557312
y: -0.05036485567688942
z: -0.6991325616836548
visibili

0 x: 0.5909948945045471
y: -0.09403908997774124
z: -0.53358393907547
visibility: 0.997015655040741

1 x: 0.5969868898391724
y: -0.10792586952447891
z: -0.5607901811599731
visibility: 0.9931755661964417

2 x: 0.5988150835037231
y: -0.10914061963558197
z: -0.5606633424758911
visibility: 0.9916678667068481

3 x: 0.6008186936378479
y: -0.11006037145853043
z: -0.5607793927192688
visibility: 0.9918984770774841

4 x: 0.5905447006225586
y: -0.10735402256250381
z: -0.5689619779586792
visibility: 0.9956585764884949

5 x: 0.5883234739303589
y: -0.10811176151037216
z: -0.5692023038864136
visibility: 0.9953065514564514

6 x: 0.5857064723968506
y: -0.10895678400993347
z: -0.569412350654602
visibility: 0.9961752891540527

7 x: 0.603019118309021
y: -0.12194399535655975
z: -0.5314216017723083
visibility: 0.993625283241272

8 x: 0.5728964805603027
y: -0.12081816047430038
z: -0.5643184781074524
visibility: 0.9971740245819092

9 x: 0.5854735374450684
y: -0.08564107120037079
z: -0.5024301409721375
visibili

0 x: 0.5548673272132874
y: -0.10857365280389786
z: -0.6026064157485962
visibility: 0.963168203830719

1 x: 0.5609720349311829
y: -0.126066654920578
z: -0.6111931800842285
visibility: 0.9227940440177917

2 x: 0.5638822317123413
y: -0.12642544507980347
z: -0.6112204194068909
visibility: 0.9189155697822571

3 x: 0.5669084191322327
y: -0.12658478319644928
z: -0.6112954020500183
visibility: 0.9112189412117004

4 x: 0.5518895387649536
y: -0.12783698737621307
z: -0.6200681924819946
visibility: 0.9488649964332581

5 x: 0.5486055016517639
y: -0.12938426434993744
z: -0.6203142404556274
visibility: 0.9513727426528931

6 x: 0.545148491859436
y: -0.13097968697547913
z: -0.6208040714263916
visibility: 0.9556893110275269

7 x: 0.5700299739837646
y: -0.13210803270339966
z: -0.4921504855155945
visibility: 0.9194133281707764

8 x: 0.5350826978683472
y: -0.13990361988544464
z: -0.5233852863311768
visibility: 0.9660334587097168

9 x: 0.5542841553688049
y: -0.09439459443092346
z: -0.5452591180801392
visibi

0 x: 0.4922395646572113
y: 0.16076084971427917
z: -0.44040602445602417
visibility: 0.9598178863525391

1 x: 0.49930962920188904
y: 0.15204888582229614
z: -0.47497692704200745
visibility: 0.910590648651123

2 x: 0.5041186809539795
y: 0.14938394725322723
z: -0.4748581051826477
visibility: 0.9067031145095825

3 x: 0.5089151263237
y: 0.14697250723838806
z: -0.4749477803707123
visibility: 0.8906970024108887

4 x: 0.4865536093711853
y: 0.15374231338500977
z: -0.47480708360671997
visibility: 0.9446426630020142

5 x: 0.48263248801231384
y: 0.15223568677902222
z: -0.475014865398407
visibility: 0.9491163492202759

6 x: 0.478689044713974
y: 0.15074226260185242
z: -0.47531524300575256
visibility: 0.9534652233123779

7 x: 0.5242332220077515
y: 0.12220616638660431
z: -0.4694584906101227
visibility: 0.8999156951904297

8 x: 0.47115153074264526
y: 0.11953213810920715
z: -0.42959505319595337
visibility: 0.9676377177238464

9 x: 0.500445544719696
y: 0.14890247583389282
z: -0.41510167717933655
visibility

0 x: 0.5448434352874756
y: 0.028669623658061028
z: -0.3760005235671997
visibility: 0.9385156631469727

1 x: 0.5550386309623718
y: 0.015315412543714046
z: -0.409229576587677
visibility: 0.8703556060791016

2 x: 0.5589280128479004
y: 0.013196934014558792
z: -0.40911293029785156
visibility: 0.8631308674812317

3 x: 0.5628679990768433
y: 0.011283477768301964
z: -0.4091358184814453
visibility: 0.8466807007789612

4 x: 0.5445833206176758
y: 0.01577412337064743
z: -0.4108431339263916
visibility: 0.9218330979347229

5 x: 0.5412715077400208
y: 0.013750921003520489
z: -0.4111826419830322
visibility: 0.9266542196273804

6 x: 0.5379010438919067
y: 0.011701960116624832
z: -0.4116746485233307
visibility: 0.936691403388977

7 x: 0.5744680166244507
y: -0.0156061090528965
z: -0.3931295573711395
visibility: 0.8603260517120361

8 x: 0.5320332050323486
y: -0.017146702855825424
z: -0.40160995721817017
visibility: 0.9634138941764832

9 x: 0.5479856729507446
y: 0.024809615686535835
z: -0.34851178526878357
vi